In [1]:
#Script by Brian Shotwell, G2 2022
#Tool to add five fields: "FES_ADDR_FL", "ASSET_ID_PGE_FL", "UTA_TYPE_PGE_FL", "Delivered_Date", and "Delivered_To" to the UTA points Layer and field calculate data
#Tool also does initial XY points from CSV and reprojects points to UTM Zone 10N. Also formats final output FC to AGOL schema.

import os
import arcpy
import datetime

arcpy.env.addOutputsToMap = True
arcpy.env.overwriteOutput = True

# Update uta_points variable with the latest CSV of the UTAs
# Update delivered_date to the deliverable day
# Update date to the 15th of the month day that is the deliverable day, ex: "UTA_20220715"

uta_point_csv = 'Sewer Defect Access Tracking Log_20230415.csv'
date_delivered = '2023-04-15 00:00:00'
date = '20230415'

<h4> Use XY Table to Point tool to create point feature class from CSV, then reproject the points to UTM Zone 10N </h4>

In [2]:
# Update the output name feature class with UTA_{date}
arcpy.management.XYTableToPoint(uta_point_csv, "UTA_FC", "Longitude", "Latitude", None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')
arcpy.management.Project("UTA_FC", "UTA_"+ date +"_10N", 'PROJCS["NAD_1983_UTM_Zone_10N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-123.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', "WGS_1984_(ITRF00)_To_NAD_1983", 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")
arcpy.management.Delete("UTA_FC")

<Result 'true'>

<h4> Select all the records NOT in this deliverable month and remove them from the feature class </h4>

In [3]:
uta_point_fc = "UTA_"+ date +"_10N"
arcpy.env.overwriteOutput = True

# Select all records not in this deliverable month and remove them from the feature class
arcpy.SelectLayerByAttribute_management(uta_point_fc, "NEW_SELECTION", "Date_Informed_PGE <> timestamp '" + str(date_delivered) + "'", None)
arcpy.DeleteFeatures_management(uta_point_fc)
arcpy.management.SelectLayerByAttribute(uta_point_fc, "CLEAR_SELECTION")

<Result 'UTA_20230415_10N'>

<h4> !!! Before doing the next step, double check the amount of records equals the total UTA number being delivered this month !!!</h4>

In [4]:
#AddField(in_table, field_name, field_type, {field_precision}, {field_scale}, {field_length}, {field_alias}, {field_is_nullable}, {field_is_required}, {field_domain})

arcpy.AddField_management(uta_point_fc, "FES_ADDR_FL", "TEXT", "", "", field_length = "75", field_is_nullable = "NULLABLE")
arcpy.AddField_management(uta_point_fc, "ASSET_ID_PGE_FL", "TEXT", "", "", field_length = "50", field_is_nullable = "NULLABLE")
arcpy.AddField_management(uta_point_fc, "UTA_TYPE_PGE_FL", "TEXT", "", "", field_length = "50", field_is_nullable = "NULLABLE")
arcpy.AddField_management(uta_point_fc, "Delivered_Date", "DATE", field_is_nullable = "NULLABLE")
arcpy.AddField_management(uta_point_fc, "Delivered_To", "TEXT", "", "", field_length = "50", field_is_nullable = "NULLABLE")

#CalculateField data
#CalculateField(in_table, field, expression, {expression_type}, {code_block}, {field_type})

#str_addr = !Street_Number! + " " + !Street_Name! + " " + !Street_Suffix!
arcpy.CalculateField_management(uta_point_fc, "FES_ADDR_FL", ("!Street_Number!" + "' '" + "!Street_Name!" + "' '" + "!Street_Suffix!").upper(), "PYTHON_9.3")
arcpy.CalculateField_management(uta_point_fc, "ASSET_ID_PGE_FL", "!Asset_Number!", "PYTHON_9.3")
arcpy.CalculateField_management(uta_point_fc, "UTA_TYPE_PGE_FL", "!UTA_Type!", "PYTHON_9.3")
arcpy.CalculateField_management(uta_point_fc, "Delivered_Date", "!Date_Informed_PGE!", "PYTHON_9.3")

#Select records that don't have a street suffix and calculate them over to FES_ADDR_FL
arcpy.management.SelectLayerByAttribute(uta_point_fc, "NEW_SELECTION", "FES_ADDR_FL IS NULL", None)
arcpy.CalculateField_management(uta_point_fc, "FES_ADDR_FL", ("!Street_Number!" + "' '" + "!Street_Name!"), "PYTHON_9.3")
arcpy.management.SelectLayerByAttribute(uta_point_fc, "CLEAR_SELECTION")

#Upper case all the Addresses
arcpy.management.SelectLayerByAttribute(uta_point_fc, "NEW_SELECTION", "FES_ADDR_FL IS NOT NULL", None)
arcpy.management.CalculateField(uta_point_fc, "FES_ADDR_FL", "!FES_ADDR_FL!.upper()", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

#Calculate Delivered_To fields to be either PG&E or PG&E/SFPUC
arcpy.management.SelectLayerByAttribute(uta_point_fc, "NEW_SELECTION", "Date_Informed_City IS NOT NULL", None)
arcpy.management.CalculateField(uta_point_fc, "Delivered_To", "'PG&E/SFPUC'", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
arcpy.management.SelectLayerByAttribute(uta_point_fc, "NEW_SELECTION", "Delivered_To IS NULL", None)
arcpy.management.CalculateField(uta_point_fc, "Delivered_To", "'PG&E'", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
arcpy.management.SelectLayerByAttribute(uta_point_fc, "CLEAR_SELECTION")

#Title case all the UTA_Type_PGE_FL values
arcpy.management.CalculateField(uta_point_fc, "UTA_TYPE_PGE_FL", "!UTA_TYPE_PGE_FL!.title()", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

#Delete un-wanted fields
arcpy.DeleteField_management(uta_point_fc,['UTA_ID','FES_ADDR_FL','ASSET_ID_PGE_FL','UTA_TYPE_PGE_FL','Delivered_Date','Delivered_To'], "KEEP_FIELDS")

#Find and replace on all 'P Trap' values with 'P-Trap'
arcpy.management.CalculateField(uta_point_fc, "UTA_TYPE_PGE_FL", "!UTA_TYPE_PGE_FL!.replace('P Trap', 'P-Trap')", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField(uta_point_fc, "UTA_TYPE_PGE_FL", "!UTA_TYPE_PGE_FL!.replace('Ptrap', 'P-Trap')", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

<Result 'UTA_20230415_10N'>